In [21]:
!tar -xzvf my.tar.gz

my/
my/weights.index
my/weights.data-00000-of-00001
my/checkpoint


In [22]:
import io
import os
import re
import shutil
import string
import numpy as np
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Create a custom standardization function to strip HTML break tags '<br />'.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
  return tf.strings.regex_replace(stripped_html,
                                  '[%s]' % re.escape(string.punctuation), '')


# Vocabulary size and number of words in a sequence.
vocab_size = 10000
sequence_length = 280 # 100

# Use the text vectorization layer to normalize, split, and map strings to
# integers. Note that the layer uses the custom standardization defined above.
# Set maximum_sequence length as all samples are not of the same length.
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

embedding_dim=16

my_model = Sequential([
  vectorize_layer,
  Embedding(vocab_size, embedding_dim, name="embedding"),
  GlobalAveragePooling1D(),
  Dense(16, activation='relu'),
  Dense(3) # category count
])

my_model.load_weights('my/weights')

0 - hate speech 1 - offensive language 2 - neither

In [23]:
y_pred = my_model.predict(['sun is shining'])[0]
y_pred = np.argmax(y_pred, axis=-1)
y_pred = y_pred.squeeze()
y_pred
# 0 - hate speech 1 - offensive language 2 - neither

2